In [1]:
%reset -f
import random
from Basketball_Dataset_rnn import Dataset
import pandas as pd
from collections import deque
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
SEQ_LEN = 3

In [3]:
dataset_20150320 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20150320.csv', "20150320", home_visit = 1, save = True)

dataset20150320 is imported.


In [4]:
def build_model(training_X):
#     print(training_X).shape
    model = Sequential()
    model.add(LSTM(128, input_shape=((SEQ_LEN, 49)), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())

    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(6, activation='softmax'))
    return model

In [5]:
def feature_label_splitting(dataset):
    X = []
    y = []

    for seq, target in dataset:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)

    return np.array(X), y  # return X and y...and make X a numpy array!


In [6]:
def predict(dataframe, chosen_model):
    testing_set, _ = make_sequence_set(dataframe)
    testing_X, testing_y = feature_label_splitting(testing_set)
    model = build_model(testing_X)
    opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.load_weights(chosen_model)
    probs = model.predict(testing_X)
    return probs

In [7]:
def make_sequence_set(dataframe):
    first = 0
    second = 1
    sequential_data = []
    row_nums = []
    while first < dataframe.shape[0] or second < dataframe.shape[0]:
        ready_stop = False
        while ((second < dataframe.shape[0]) and ((dataframe["shot.clock"].iloc[second] < 0.9999) or (dataframe["shot.clock"].iloc[second] > 0.9999 and ready_stop == False))):
            if dataframe["shot.clock"].iloc[second] < 0.9999:
                ready_stop = True
            second += 1
        start_ind = dataframe["shot.clock"].iloc[[first]].index[0]
        end_ind = dataframe["shot.clock"].iloc[[second - 2]].index[0]
        start_iloc_ind = first
        end_iloc_ind = second - 2
#         temp_frame = dataframe.iloc[start_iloc_ind: end_iloc_ind,:]
#         saveline = "temp/temp" + str(first) + ".csv"
#         temp_frame.to_csv(saveline)
        sequence = deque(maxlen=SEQ_LEN)
        i = start_iloc_ind
        for row in dataframe.iloc[start_iloc_ind: end_iloc_ind,:].values:
            sequence.append([n for n in row[:-1]])
            if len(sequence) == SEQ_LEN:
                sequential_data.append([np.array(sequence), row[-1]])
                row_nums.append(i)
            i += 1
        first = second
        second += 1
#     random.shuffle(sequential_data)
    return sequential_data, row_nums


In [8]:
sequential_data, row_nums = make_sequence_set(dataset_20150320.nn_ready)

In [9]:
probs = predict(dataset_20150320.nn_ready, "RNN_cross/cross_18/cp-0017.ckpt")

W0726 11:46:34.700086 4477543872 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1628: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
prediction = []
for i in range(probs.shape[0]):
    prob_vector = probs[i].tolist()
#     print(prob_vector.index(max(prob_vector)))
    prediction.append(prob_vector.index(max(prob_vector)))
#     print(type(prob_vector))
true_label = []
for element in sequential_data:
    true_label.append(element[1])


In [11]:
erro_list = [0] * 6
for i in range(len(true_label)):
    if true_label[i] != prediction[i]:
        erro_list[int(true_label[i])] += 1
print(erro_list)

[21, 37, 202, 17, 1, 5]


In [12]:
print(1 - (sum(erro_list) / len(true_label)))

0.7651452282157676


In [14]:
label_list = ['touch', 'dribble', 'pass', "shot", "rebound", "turnover"]
for erro in erro_list:
    printline = "The error rate of " + label_list[erro_list.index(erro)] + " is " + str(round(erro / sum(erro_list), 2))
    print(printline)

The error rate of touch is 0.07
The error rate of dribble is 0.13
The error rate of pass is 0.71
The error rate of shot is 0.06
The error rate of rebound is 0.0
The error rate of turnover is 0.02


In [15]:
erro_list = [0] * 6
for i in range(len(true_label)):
    if true_label[i] != prediction[i]:
        erro_list[int(true_label[i])] += 1
print(erro_list)

[21, 37, 202, 17, 1, 5]


In [16]:
prediction_labels = []
row_limit_ind = 0
for i in range(dataset_20150320.nn_ready.shape[0]):
#     if i >= len(row_nums):
#         print("i",i)
#         prediction_labels.append(-1)
#         continue
#     if i != row_nums[row_limit_ind]:
#         print("i2", i)
#         prediction_labels.append(-1)
#     else:
# #         print("i",i)
# #         print("???", i == row_nums[row_limit_ind])
#         print("row_nus", row_limit_ind)
#         prediction_labels.append(prediction[row_limit_ind])
#         row_limit_ind += 1
    if row_limit_ind < len(row_nums) and i == row_nums[row_limit_ind]:
        prediction_labels.append(prediction[row_limit_ind])
        row_limit_ind += 1
    else:
        prediction_labels.append(-1)

In [17]:
print(len(prediction_labels))

1453


In [18]:
from Basketball_Dataset_rnn import Dataset
from Basketball_set_funcitons_new import reverse_dict, loaded_prediction
from Basketball_model_copy import Basketball_model

In [19]:
# erro_list = [0] * 6
# for i in range(len(prediction_labels)):
#     if true_label[i] != prediction[i]:
#         erro_list[int(true_label[i])] += 1
# print(erro_list)
without_minus_1 = []
for element in prediction_labels:
    if element == -1:
        continue
    without_minus_1.append(element)

In [20]:
labels = [0] * 6
for element in prediction_labels:
    labels[element] += 1
print(labels)

[286, 792, 116, 0, 11, 248]


In [21]:
print(len(prediction_labels))
print(dataset_20150320.nn_ready.shape[0])
print(len(without_minus_1))
print(len(true_label))

1453
1453
1205
1205


In [22]:
erro_list = [0] * 6
for i in range(len(true_label)):
    if true_label[i] != without_minus_1[i]:
        erro_list[int(true_label[i])] += 1
print(erro_list)

[21, 37, 202, 17, 1, 5]


In [23]:
dataset_20150320.nn_ready

,game.clock,shot.clock,event.descrip_0,event.descrip_1,event.descrip_2,event.descrip_3,event.descrip_4,event.descrip_5,event.descrip_6,position_0,...,p.poss_2,p.poss_3,p.poss_4,p.poss_5,p.poss_6,p.poss_7,p.poss_8,p.poss_9,score_difference,label
34,0.976522,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,2
35,0.975776,1.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,0
36,0.975640,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,1
37,0.974971,0.994213,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,1
38,0.974360,0.973380,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,1
39,0.973487,0.943287,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,1
40,0.972571,0.912037,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,2
41,0.972105,0.895544,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,0
42,0.971766,0.883970,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,1
43,0.970985,0.857350,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,2


In [24]:
dataset_20150320.nn_ready["prediction"] = prediction_labels


W0726 11:49:18.265901 4477543872 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0726 11:49:18.266671 4477543872 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0726 11:49:18.267470 4477543872 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0726 11:49:18.268169 4477543872 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0726 11:49:18.268943 4477543872 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0726 11:49:18.269792 4477543872 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-1.gamma
W0726 11:49:18.270568 4477543872 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-1.beta
W0726 11:49:18.271275 4477543872 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-3.gamma
W0726 11:49:18.274333 4477543872 util.py:144

In [25]:
dataset_20150320.nn_ready.to_csv("rnn.csv")

In [26]:
dataset_20150320.nn_ready

,game.clock,shot.clock,event.descrip_0,event.descrip_1,event.descrip_2,event.descrip_3,event.descrip_4,event.descrip_5,event.descrip_6,position_0,...,p.poss_3,p.poss_4,p.poss_5,p.poss_6,p.poss_7,p.poss_8,p.poss_9,score_difference,label,prediction
34,0.976522,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,2,-1
35,0.975776,1.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,0,-1
36,0.975640,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,1,1
37,0.974971,0.994213,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,1,1
38,0.974360,0.973380,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,1,1
39,0.973487,0.943287,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,1,1
40,0.972571,0.912037,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,2,1
41,0.972105,0.895544,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,0,0
42,0.971766,0.883970,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,1,1
43,0.970985,0.857350,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,2,1


In [27]:
new_dataset = dataset_20150320.nn_ready[dataset_20150320.nn_ready["prediction"] != -1]

In [28]:
new_dataset = new_dataset.drop(columns = ["prediction"])

In [29]:
sample = new_dataset.iloc[:].values
a = loaded_prediction("../saved_models/model_crossvalidated_final", sample, reverse_dict(dataset_20150320.real_label_number), mode = "more", verbose = False)


Accuracy is: 75.850622406639%


In [30]:
label_list = {'touch': 0 , 'dribble' : 1, 'pass': 2, "shot": 3, "rebound": 4, "turnover": 5}
prediction_label = a[1]
prediction_label_num = []
for element in prediction_label:
    prediction_label_num.append(label_list[element])

In [32]:
true_label =new_dataset["label"].values
erro_list = [0] * 6
for i in range(len(true_label)):
    if true_label[i] != prediction_label_num[i]:
        erro_list[int(true_label[i])] += 1
print(erro_list)
label_list = ['touch', 'dribble', 'pass', "shot", "rebound", "turnover"]
for erro in erro_list:
    printline = "The error rate of " + label_list[erro_list.index(erro)] + " is " + str(round(erro / sum(erro_list), 2))
    print(printline)

[15, 30, 224, 15, 2, 5]
The error rate of touch is 0.05
The error rate of dribble is 0.1
The error rate of pass is 0.77
The error rate of touch is 0.05
The error rate of rebound is 0.01
The error rate of turnover is 0.02


In [254]:
new_dataset_rnn = new_dataset.copy()

In [256]:
new_dataset["prediction"] = prediction_label_num

In [258]:
new_dataset.to_csv("snn.csv")

In [ ]:
dataset_20150320.nn_ready.to_csv("rnn_for_shinny.csv")

In [55]:
a = (probs[0].tolist())
print(type(a))
print(type(probs[0].tolist()))

<class 'list'>
<class 'list'>


In [26]:
for element in sequential_data:
    tru
    print(element[1])

1.0
1.0
1.0
1.0
2.0
0.0
1.0
2.0
0.0
2.0
0.0
2.0
0.0
1.0
2.0
0.0
1.0
1.0
1.0
2.0
0.0
0.0
2.0
0.0
1.0
2.0
0.0
1.0
1.0
1.0
5.0
0.0
1.0
1.0
1.0
1.0
2.0
0.0
2.0
0.0
2.0
0.0
1.0
1.0
1.0
1.0
2.0
0.0
2.0
0.0
2.0
0.0
1.0
2.0
0.0
1.0
1.0
1.0
1.0
1.0
2.0
0.0
2.0
0.0
2.0
0.0
1.0
1.0
1.0
1.0
1.0
2.0
0.0
2.0
0.0
1.0
1.0
1.0
1.0
1.0
2.0
0.0
1.0
1.0
1.0
1.0
1.0
2.0
0.0
1.0
1.0
2.0
0.0
2.0
0.0
3.0
0.0
1.0
2.0
0.0
1.0
1.0
2.0
0.0
0.0
1.0
1.0
3.0
0.0
1.0
1.0
1.0
1.0
2.0
0.0
2.0
0.0
2.0
0.0
1.0
1.0
2.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
0.0
2.0
0.0
2.0
0.0
2.0
0.0
1.0
1.0
1.0
1.0
2.0
0.0
1.0
1.0
1.0
1.0
2.0
0.0
1.0
1.0
1.0
2.0
0.0
1.0
1.0
2.0
0.0
1.0
1.0
1.0
1.0
1.0
2.0
0.0
1.0
1.0
1.0
1.0
1.0
2.0
0.0
2.0
0.0
2.0
0.0
2.0
0.0
2.0
0.0
1.0
1.0
2.0
0.0
2.0
0.0
1.0
1.0
1.0
2.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
2.0
0.0
2.0
0.0
1.0
2.0
0.0
1.0
1.0
2.0
0.0
2.0
0.0
2.0
0.0
2.0
0.0
2.0
0.0
3.0
0.0
1.0
1.0
1.0
2.0
0.0
2.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
0.0
1.0
1.0
1.0
5.0
2.0
0.0
1.0
1.0
2.0


In [29]:
for row_num in row_nums:
    print(row_num)

TypeError: object of type 'int' has no len()

In [31]:
print(len(row_nums))

1205


In [8]:
row = dataset_20150320.nn_ready.iloc[[2]]

In [14]:
i = 0
for row in dataset_20150320.nn_ready.iloc[0: 12,:].values:
    print(i)
    i += 1

0
1
2
3
4
5
6
7
8
9
10
11
